In [22]:
import nltk
from nltk.corpus import sentiwordnet as swn
from pycorenlp import StanfordCoreNLP
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
import requests
import sys, os
import numpy as np
import pandas as pd

In [23]:
nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

In [24]:
breakdown = swn.senti_synset('good.n.01')
print(breakdown.pos_score())

0.5


In [25]:
wn.synsets('love')

[Synset('love.n.01'),
 Synset('love.n.02'),
 Synset('beloved.n.01'),
 Synset('love.n.04'),
 Synset('love.n.05'),
 Synset('sexual_love.n.02'),
 Synset('love.v.01'),
 Synset('love.v.02'),
 Synset('love.v.03'),
 Synset('sleep_together.v.01')]

In [26]:
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

In [27]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

def read_file(file):
    f = open(file, 'r')

    pattern_title = '\[t\]'
    pattern_sentence = '(?<=##).+'
    pattern_aspect = '.+(?=##)'

    review = []
    for a in f:
        if re.search('##', a):
            sentence = re.findall(pattern_sentence, a)[0]
            aspect = re.findall(pattern_aspect, a)
            if len(aspect) > 0:
                aspect = aspect[0]
            else:
                aspect = ''
            review.append((sentence, aspect))

    df = pd.DataFrame(columns=['review','target'])
    for r in review:
        df = df.append({'review': r[0], 'target': r[1]}, ignore_index=True)
        
    return df

In [28]:
def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append((pos['word'], pos['pos']))
    return res

In [29]:
def get_score(sentence, key, tagged_sentence):
    flag = True
    
    tagged = ('','')
    for p in tagged_sentence:
        if p[0] == key:
            tagged = p
    if tagged == ('', ''):
        flag = False

    ambiguous = tagged[0]
    tag = tagged[1]
    pos = ''

    if 'NN' in tag:
        pos = 'n'
    elif 'NNS' in tag:
        pos = 'nns'
    elif 'VB' in tag:
        pos = 'v'
    elif 'VBG' in tag:
        pos = 'v'
    elif 'JJ' in tag:
        pos = 'a'
    elif 'RB' in tag:
        pos = 'r'
    else:
        flag = False

    endscore = 0
    if flag:
        answer = cosine_lesk(sentence, ambiguous, pos)
        if answer:
            score = swn.senti_synset(answer.name())

            if score.pos_score() > score.neg_score():
                endscore = score.pos_score()
            else:
                endscore = score.neg_score() * (-1)
    return endscore

In [30]:
def get_sentence_score(sentence):
    scores = []
    tagged_sentence = pos_tag(sentence)
    for word in sentence.split(' '):
#         if word in op_set:
        score = get_score(sentence, word, tagged_sentence)
        scores.append(score)
    if len(scores):
        return np.mean(scores)
    return 0

In [31]:
def main_program(review, target, filename):
    scores = []
    labels = []
    index = 0
    for sentence in review:
        aspects = target[index].split(',')
        label = 0
        if aspects[0] != '':
            for aspect in aspects:
                try:
                    tanda = aspect.split('[')[1][0]
                    angka = aspect.split('[')[1][1]
                    if tanda == '+':
                        label += int(angka)
                    else:
                        label -= int(angka)
                except:
                    pass
#                     print(aspect)

            score = get_sentence_score(sentence)

            if label > 0:
                labels.append(1)
            elif label <= 0:
                labels.append(2)
    #         else:            
    #             labels.append(0)

            if score > 0:
                scores.append(1)
            elif score <= 0:
                if(score == 0):
                    print(aspects, sentence)
                scores.append(2)
    #         else:            
    #             scores.append(0)
            
        index += 1
    
#     data = { 'review':review, 'label': labels, 'prediction': scores}
#     out = pd.DataFrame(data)
#     out.to_csv(filename)
    
    return labels, scores

In [32]:
df = read_file('dataset/bing_liu/Nikon coolpix 4300.txt')

In [33]:
label, pred = main_program(df['review'], df['target'], 'sentiwordnet.csv')

['transfer[-2]'] 1 . pictures wo n't transfer to pc directly from the camera using the included transfer cable . 
['auto mode[-2]'] . this camera keeps on autofocussing in auto mode with a buzzing sound which can 't be stopped . 
['image[-2]'] . even the slightest of the shakes totally distorts your image 
['indoor image[-2]'] . images taken indoor were n't so clear . 
['lens cap[-3]'] . lens cap is a really annoying 
['movie[-2]'] . the movie clips taken will always have some ' noise ' in it - you can 't avoid that . 
['indoor picture[-1]'] 1 . pictures taken in an indoor setting are a little dull and rarely hazy if you tend to take photographs from a distance . 
['delay[-2][u]'] 2 . it takes a while for the camera to actually capture the photograph from the time you click the button and i have seen people becoming a little impatient waiting for the flash to glow . 
['picture[-2]'] 3 . the pictures come out hazy if your hands shake even for a moment during the entire process of taking

In [34]:
y_actu = pd.Series(label, name='Actual')
y_pred = pd.Series(pred, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)

In [36]:
df_confusion

Predicted,1,2
Actual,,
1,94,35
2,8,23


In [15]:
from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(label, pred)

In [18]:
cm.print_stats()

population: 0
P: 0
N: 0
PositiveTest: 0
NegativeTest: 0
TP: 0
TN: 0
FP: 0
FN: 0
TPR: nan
TNR: nan
PPV: nan
NPV: nan
FPR: nan
FDR: nan
FNR: nan
ACC: nan
F1_score: nan
MCC: nan
informedness: nan
markedness: nan
prevalence: nan
LRP: nan
LRN: nan
DOR: nan
FOR: nan


C:\ProgramData\Anaconda3\lib\site-packages\pandas_ml\confusion_matrix\bcm.py:191: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.TP) / self.P)
C:\ProgramData\Anaconda3\lib\site-packages\pandas_ml\confusion_matrix\bcm.py:213: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.TN) / self.N)
C:\ProgramData\Anaconda3\lib\site-packages\pandas_ml\confusion_matrix\bcm.py:236: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.TP) / self.PositiveTest)
C:\ProgramData\Anaconda3\lib\site-packages\pandas_ml\confusion_matrix\bcm.py:259: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.TN) / self.NegativeTest)
C:\ProgramData\Anaconda3\lib\site-packages\pandas_ml\confusion_matrix\bcm.py:181: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.FP) / self.N)
C:\ProgramData\Anaconda3\lib\site-packages\pandas_ml\confusion_matrix\bcm

In [17]:
len(label)

160

In [62]:
get_sentence_score('great camera , great investment ! ')

great -0.0
camera -0.0
, 0
great -0.0
investment -0.0
! 0
 0


0.0